In [13]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import torch
import torchvision.transforms as transforms
from facenet_pytorch import MTCNN,InceptionResnetV1
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class FaceLoading:
    def __init__(self,directory):
        self.directory = directory
        self.target_size = (112, 112)
        self.X = []
        self.y = []
        self.mtcnn = MTCNN(image_size=160, margin=0, min_face_size=20,
            thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True)
        self.i = 1
    def extract_face(self,path):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        face = self.mtcnn(img)
        return face
    def load_face_and_class(self):
        for sub_dir in os.listdir(self.directory):
            sub_dir_path = os.path.join(self.directory, sub_dir)
            for img_name in os.listdir(sub_dir_path):
                face = self.extract_face(os.path.join(sub_dir_path, img_name))
                if(face is not None):
                    self.X.append(face)
                    self.y.append(sub_dir)
        return np.array(self.X), np.array(self.y)
    def plot_images(self):
        num_columns = 3
        num_rows = math.ceil(len(self.X) / num_columns)  # Ensure row count covers all images

        plt.figure(figsize=(num_columns * 3, num_rows * 3))  # Adjust figure size dynamically
        for num, img in enumerate(self.X):
            plt.subplot(num_rows, num_columns, num + 1)
            plt.imshow(img.permute(1, 2, 0).numpy())
            plt.axis('off')
        plt.tight_layout()  # Improve layout spacing
        plt.show()
face_loading = FaceLoading(r"C:\Users\admin\OneDrive - Hanoi University of Science and Technology\Documents\GitHub\PTTK\face_recognization\source\data_raw\image")
X,y = face_loading.load_face_and_class()
encode = LabelEncoder()
encoded_y = encode.fit_transform(y)
encoded_y = encoded_y.reshape(-1, 1)
onehot_encoder = OneHotEncoder(sparse_output=False)  # sparse_output=False để trả về mảng NumPy
onehot_y = onehot_encoder.fit_transform(encoded_y)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Assuming resnet is your InceptionResnetV1 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet = resnet.to(device)
resnet.eval()  # Set model to evaluation mode

# Preprocessing pipeline
preprocess = transforms.Compose([
    transforms.Resize((160, 160)),  # Resize to 160x160
    transforms.ToTensor(),          # Converts to (C, H, W) and normalizes to [0, 1]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

X_embed = []
for face in X:
    if isinstance(face, np.ndarray):
        # Check shape and type
        # print(f"Shape: {face.shape}, Dtype: {face.dtype}")

        # Ensure face is a valid image array
        if face.ndim == 3:
            if face.shape[-1] in [1, 3]:  # (H, W, C) format
                if face.shape[-1] == 1:  # Convert grayscale to RGB
                    face = np.repeat(face, 3, axis=-1)
            elif face.shape[0] in [1, 3]:  # (C, H, W) format
                face = face.transpose(1, 2, 0)  # Convert to (H, W, C)
                if face.shape[-1] == 1:  # Convert grayscale to RGB
                    face = np.repeat(face, 3, axis=-1)
            else:
                raise ValueError(f"Invalid face shape: {face.shape}. Expected (H, W, 3), (H, W, 1), or (3, H, W).")
        elif face.ndim == 2:  # Grayscale (H, W)
            face = np.stack([face] * 3, axis=-1)  # Convert to RGB
        else:
            raise ValueError(f"Invalid face shape: {face.shape}. Expected (H, W, 3), (H, W, 1), or (H, W).")

        # Ensure uint8 for PIL (if float, convert to uint8)
        if face.dtype != np.uint8:
            face = (face * 255).clip(0, 255).astype(np.uint8)

        face_pil = Image.fromarray(face)

        # Apply preprocessing
        face_tensor = preprocess(face_pil).unsqueeze(0).to(device)  # Shape: [1, 3, 160, 160]

        # Get embedding
        with torch.no_grad():
            embedding = resnet(face_tensor).cpu().detach().numpy()
        X_embed.append(embedding)
    else:
        raise ValueError("Each face in X should be a NumPy array")

X_embed = np.array(X_embed)

# Lưu embedding và nhãn
print("Đang lưu embedding và nhãn...")

# Reshape embedding để loại bỏ dimension thừa
X_embed = X_embed.reshape(X_embed.shape[0], -1)

np.save('face_embeddings.npy', X_embed)
print(f"Đã lưu embedding với shape: {X_embed.shape}")

np.save('face_labels.npy', y)
print(f"Đã lưu nhãn gốc: {len(y)} samples")


Đang lưu embedding và nhãn...
Đã lưu embedding với shape: (87, 512)
Đã lưu nhãn gốc: 87 samples


In [15]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra (3:1)
X_train, X_test, y_train, y_test = train_test_split(X_embed, y, test_size=0.25, stratify=y, random_state=42)

# Chuyển đổi nhãn thành số nguyên
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Tính độ tương đồng cosine giữa các embedding trong tập kiểm tra
cos_sim_matrix = cosine_similarity(X_test)

# Khởi tạo danh sách để lưu nhãn dự đoán
y_pred = []

# Đặt ngưỡng 70% (0.7) cho độ tương đồng cosine
threshold = 0.7

# So sánh từng cặp embedding trong tập kiểm tra
for i in range(len(X_test)):
    max_sim = -1
    predicted_label = y_test_encoded[i]  # Mặc định là nhãn gốc nếu không tìm thấy cặp tương đồng
    for j in range(len(X_test)):
        if i != j and cos_sim_matrix[i][j] > max_sim and cos_sim_matrix[i][j] >= threshold:
            max_sim = cos_sim_matrix[i][j]
            predicted_label = y_test_encoded[j]
    y_pred.append(predicted_label)

# Tính các chỉ số đánh giá
accuracy = accuracy_score(y_test_encoded, y_pred)
precision = precision_score(y_test_encoded, y_pred, average='weighted')
recall = recall_score(y_test_encoded, y_pred, average='weighted')
f1 = f1_score(y_test_encoded, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")




Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
